<a href="https://colab.research.google.com/github/alinahasss/mtsss/blob/main/mts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install datasets
!pip install transformers
!pip install evaluate torch nltk rouge_score numpy
!pip install gtts
!pip install deep_translator
!pip install pyttsx3
from gtts import gTTS
from IPython.display import Audio
import pyttsx3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
import datasets
from transformers import VisionEncoderDecoderModel, AutoFeatureExtractor,AutoTokenizer
os.environ["WANDB_DISABLED"] = "true"

import nltk
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    nltk.download("punkt", quiet=True)

In [6]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor

image_encoder_model = "google/vit-base-patch16-224-in21k"
text_decode_model = "gpt2"

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    image_encoder_model, text_decode_model)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.6.crossattention.c_proj.bias', 'h.10.crossattention.q_attn.weight', 'h.1.crossattention.c_proj.bias', 'h.4.crossattention.q_attn.weight', 'h.2.crossattention.c_attn.weight', 'h.9.crossattention.c_proj.bias', 'h.2.crossattention.c_proj.bias', 'h.3.crossattention.q_attn.weight', 'h.1.crossattention.bias', 'h.5.crossattention.bias', 'h.6.crossattention.masked_bias', 'h.4.crossattention.c_proj.weight', 'h.6.crossattention.q_attn.weight', 'h.2.crossattention.q_attn.weight', 'h.1.crossattention.masked_bias', 'h.4.crossattention.c_proj.bias', 'h.9.crossattention.bias', 'h.0.crossattention.c_proj.bias', 'h.11.crossattention.q_attn.weight', 'h.7.crossattention.c_attn.weight', 'h.7.crossattention.c_proj.bias', 'h.11.crossattention.masked_bias', 'h.9.ln_cross_attn.weight', 'h.6.crossattention.c_proj.weight', 'h.1.crossattention.c_attn.weight', 'h.5.crossattention.c_proj.bias', 'h.

In [7]:
feature_extractor = AutoFeatureExtractor.from_pretrained(image_encoder_model)
tokenizer = AutoTokenizer.from_pretrained(text_decode_model)

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [12]:
tokenizer.pad_token = tokenizer.eos_token

model.config.eos_token_id = tokenizer.eos_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

In [13]:
output_dir = "vit-gpt-model"
model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('vit-gpt-model/tokenizer_config.json',
 'vit-gpt-model/special_tokens_map.json',
 'vit-gpt-model/vocab.json',
 'vit-gpt-model/merges.txt',
 'vit-gpt-model/added_tokens.json',
 'vit-gpt-model/tokenizer.json')

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
COCO_DIR = ("/content/gdrive/MyDrive/mts1")
ds = datasets.load_dataset("ydshieh/coco_dataset_script", "2017", data_dir=COCO_DIR)

  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
ds
ds['train'][0]

{'image_id': 74,
 'caption_id': 145996,
 'caption': 'A picture of a dog laying on the ground.',
 'height': 426,
 'width': 640,
 'file_name': '000000000074.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000000074.jpg',
 'image_path': '/root/.cache/huggingface/datasets/downloads/extracted/6e596361be62fefcaa5ff9d32a3b3a41327320c846deb00fc374eea53d9639b6/train2017/000000000074.jpg'}

In [17]:
from PIL import Image

# text preprocessing step
def tokenization_fn(captions, max_target_length):
    labels = tokenizer(captions, 
                      padding="max_length", 
                      max_length=max_target_length).input_ids

    return labels

# image preprocessing step
def feature_extraction_fn(image_paths, check_image=True):
    model_inputs = {}

    if check_image:
        images = []
        to_keep = []
        for image_file in image_paths:
            try:
                img = Image.open(image_file)
                images.append(img)
                to_keep.append(True)
            except Exception:
                to_keep.append(False)
    else:
        images = [Image.open(image_file) for image_file in image_paths]

    encoder_inputs = feature_extractor(images=images, return_tensors="np")

    return encoder_inputs.pixel_values

def preprocess_fn(examples, max_target_length, check_image = True):
    image_paths = examples['image_path']
    captions = examples['caption']    
    
    model_inputs = {}

    model_inputs['labels'] = tokenization_fn(captions, max_target_length)
    model_inputs['pixel_values'] = feature_extraction_fn(image_paths, check_image=check_image)

    return model_inputs

In [18]:
processed_dataset = ds.map(
    function=preprocess_fn,
    batched=True,
    fn_kwargs={"max_target_length": 128},
    remove_columns=ds['train'].column_names
)
processed_dataset

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 80
    })
    validation: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 80
    })
    test: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 16
    })
})

In [19]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir="/content/gdrive/MyDrive/mts1",
)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [20]:
import evaluate
metric = evaluate.load("rouge")

In [21]:
import numpy as np

ignore_pad_token_for_loss = True


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if ignore_pad_token_for_loss:
  
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
                                                     decoded_labels)

    result = metric.compute(predictions=decoded_preds,
                            references=decoded_labels,
                            use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [24]:
from transformers import default_data_collator
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['validation'],
    data_collator=default_data_collator,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.340336,18.251700,0.603800,18.214900,18.234400,7.000000


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


KeyboardInterrupt: ignored

In [25]:
trainer.save_model("/content/gdrive/MyDrive/mts1")

In [26]:
tokenizer.save_pretrained("/content/gdrive/MyDrive/mts1")

('/content/gdrive/MyDrive/mts1/tokenizer_config.json',
 '/content/gdrive/MyDrive/mts1/special_tokens_map.json',
 '/content/gdrive/MyDrive/mts1/vocab.json',
 '/content/gdrive/MyDrive/mts1/merges.txt',
 '/content/gdrive/MyDrive/mts1/added_tokens.json',
 '/content/gdrive/MyDrive/mts1/tokenizer.json')

In [27]:
from transformers import pipeline

In [28]:
image_captioner = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [41]:
aud = image_captioner("/content/gdrive/MyDrive/mts1/giff/0001.gif")
line1 = str(aud)
import re
itog = re.sub(r'generated_text', '', line).strip()
itog

"[{'': 'a large group of people on a boat '}]"

In [44]:
from deep_translator import GoogleTranslator
to_translate = str(itog)
translated = GoogleTranslator(source='auto', target='ru').translate(to_translate)
tts = gTTS(str(translated), lang='ru')
tts.save('/content/gdrive/MyDrive/mts1/audio/1.wav')
sound_file = '1.wav'

In [63]:
aud1 = image_captioner("/content/gdrive/MyDrive/mts1/giff/0002.gif")
line2 = str(aud1)
import re
itof = re.sub(r'generated_text', '', line2).strip()
from deep_translator import GoogleTranslator
to_translate = str(itof)
translated1 = GoogleTranslator(source='auto', target='ru').translate(to_translate)
tts = gTTS(str(translated1), lang='ru')
tts.save('/content/gdrive/MyDrive/mts1/audio/2.wav')
sound_file = '2.wav'
Audio(sound_file, autoplay=True)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [45]:
!pip install pydub
from pydub import AudioSegment
song1 = AudioSegment.from_mp3("/content/gdrive/MyDrive/mts1/audio/1.wav")
song2 = AudioSegment.from_mp3("/content/gdrive/MyDrive/mts1/audio/2.wav")
beginning = song1
end = song2
without_the_middle = beginning + end
with_style = beginning.append(end)
do_it_over = song1 + song2
awesome = do_it_over.fade_in(2000).fade_out(3000)
awesome.export("/content/gdrive/MyDrive/mts1/audio/itog.mp3", format="mp3")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<_io.BufferedRandom name='/content/gdrive/MyDrive/mts1/audio/itog.mp3'>